### 1. The Enviroment

In this project we use the  MADDPG algorithm introduced in the paper [Multi-Agent Actor-Critic for Mixed
Cooperative-Competitive Environments](https://arxiv.org/pdf/1706.02275.pdf) to solve the [Tennis Enviroment](https://github.com/Unity-Technologies/ml-agents/blob/main/docs/Learning-Environment-Examples.md#tennis). 

In this environment, two agents control rackets to bounce a ball over a net. If an agent hits the ball over the net, it receives a reward of +0.1.  If an agent lets a ball hit the ground or hits the ball out of bounds, it receives a reward of -0.01.  Thus, the goal of each agent is to keep the ball in play.

The observation space consists of 8 variables corresponding to the position and velocity of the ball and racket. Each agent receives its own, local observation.  Two continuous actions are available, corresponding to movement toward (or away from) the net, and jumping. 

The task is episodic, and in order to solve the environment, your agents must get an average score of +0.5 (over 100 consecutive episodes, after taking the maximum over both agents). Specifically,

- After each episode, we add up the rewards that each agent received (without discounting), to get a score for each agent. This yields 2 (potentially different) scores. We then take the maximum of these 2 scores.
- This yields a single **score** for each episode.

The environment is considered solved, when the average (over 100 episodes) of those **scores** is at least +0.5.

All the implemantation is avaliable in the [Training Notebook](Tennis_Training.ipynb) notebook.

### 2. Learning Algorithm :  Multi-Agent Actor-Critic for Mixed Cooperative-Competitive Environments (MADDPG)

#### The Algorithm

![alt text](assets/MADDPG.png)

*Image from: [Multi-Agent Actor-Critic for Mixed
Cooperative-Competitive Environments](https://arxiv.org/pdf/1706.02275.pdf)*

MADDPG was introduced in the paper [Multi-Agent Actor-Critic for Mixed
Cooperative-Competitive Environments](https://arxiv.org/pdf/1706.02275.pdf) and was described by the actors as an adaptation of actor-critic methods that considers action policies
of other agents and is able to successfully learn policies that require complex multiagent coordination.

It was proposed to serve as general-purpose multi-agent learning
algorithm that could be applied not just to cooperative games with explicit communication channels,
but competitive games and games involving only physical interactions between agents.

MADDPG accomplish its goal by adopting the framework of centralized training with
decentralized execution. Thus, it allows the policies to use extra information to ease training, so
long as this information is not used at test time.

In short, there are N agents so will be N policies to learn. Each actor will try to estimate the best action given the agent observation. Each critic will receive the actions of all agents as long with all the states observations of all agents and will try to estimate the action value function for its agent.

The experience replay buffer contains the tuples (x, x,0, a1, . . . , aN , r1, . . . , rN ), recording
experiences of all agents.

MADDPG also uses target networks to reduce instability during the training. 

#### Pseudocode

![alt text](assets/MADDPGcode.png)

*Image from: [Multi-Agent Actor-Critic for Mixed
Cooperative-Competitive Environments](https://arxiv.org/pdf/1706.02275.pdf)*

#### Adaptation to solve the Tennis Enviroment

The code implementation in the project is an adaptation of the DDPG code used to solve the [Reacher environment](https://github.com/VictorNas/DDPG-Continuous-Control/blob/main/continuous-control-multiple/ddpg_agent.py).

To solve the enviroment two critics networks and two actors networks were trained (one for each agent). During training, each critic receives the observations of both agents along with the actions given by the actors and try to estimate the action value function of its actor. During inference, only the actors are used.

At first , to balance exploration and explotation was used **Ornstein-Uhlenbeck process** but the agent was not able to solve the enviroment with that. We are able to solve the enviroment using the **eps greedy** method.   

#### Actors Archtecture

Each **Actor** has the following archtecture:

* The input of the network is the observation vector of its agent.
* The First linear layer **fc1** is follewed by a BatchNorm Layer  **batch_fc1** and a **LeakyRelu** activation function.

* The Second linear layer **fc2** is follewed by **LeakyRelu** activation function.

* The Third linear layer **fc3** is follewed by **tanh** activation function.

#### Critic Archtecture

Each **Critic** Network has the following archtecture

* The input of the network is the observation vector of coth agents cooncatenated. Since each observation vector has size of 24, the input size is 48.

* The First linear layer **fc1** is follewed by a BatchNorm Layer  **batch_fc1** and a **LeakyRelu** activation function.

* The output of the LeakyRelu is concatenated with the actions of both agents.

* The Second linear layer **fc2** is follewed by **LeakyRelu** activation function.

* The Third linear layer **fc3** is not follewed by any activation function.

#### Chosen Hyperparameters

After some experimentation the following hyperparameters were choosen:

| hyperparameter | Value | Descrption
| --- | --- | --- |
| BUFFER_SIZE | 1e6|replay buffer size|
| BATCH_SIZE | 128|minibatch size|
| GAMMA | 0.99 |discount factor|
| TAU | 1e-3|for soft update of target parameters|
|LR_ACTOR| 3e-4|learning rate of the actor |
|LR_CRITIC|1e-3 |learning rate of the critic|
|UPDATE_STEPS|10 |update every steps|
|NUMBER_UPDATES|8|number of times we update the network after each UPDATE_STEPS|
|OPTMIZER_ACTOR|ADAM|Optimizer of the actor|
|OPTMIZER_CRITIC|ADAM|Optimizer of the critic|
|eps_init|1.0|Initial value of eps ofr the eps Greedy method|
|eps_min|0.01|Minimal value of eps ofr the eps Greedy method|
|eps_decay|0.95|Decay of the eps at each episode|

### 2.Plot of Rewards

The environment is considered solved, when the average (over 100 episodes) of those **scores** is at least +0.5.
The agent was trained for almost 3600 episodes and was able to solve the enviroment at first in episode 2417.
Near the episode 3000 the agent as able to achieve a mean score of 0.5 over the last 100 consecutives episodes.
Near the episode 3500 the agent was able to achieve a mean score of 1.51 over the last 100 consecutives episodes.

OBS: The red lines represents the goal score.

![alt text](assets/scores.png)

### Ideas for Future Work

There are some imporvements that may be worth to be done in order to make the agent to solve the enviroment faster.
1. Try to implement other algorithms like Trust Region Policy Optimization (TRPO)and Proximal Policy Optimization (PPO).
2. Try to implement the Prioritized Buffer Replay.